### Importação dos metadados

In [3]:
from src import grouper
import pandas as pd
import nltk
%load_ext autoreload
%autoreload 2
df = pd.read_csv("dados/corpus_tratado/metadados.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Filtrando os ids por assuntos

In [4]:
corte = 50
df_validos = pd.DataFrame(df.groupby("Assunto").size()).reset_index()
df_validos.columns = ["assunto", "quant"]
df_validos = df_validos.loc[df_validos.quant >= corte]
display(df_validos.quant.sum())
display(df_validos.assunto.nunique())
documentos_validos = df[df.Assunto.isin(df_validos.assunto)][["id", "Assunto"]].reset_index().drop('index', axis = 1)
documentos_validos

35027

125

,id,Assunto
0,1500075-43.2019.8.26.0569,Tráfico de Drogas e Condutas Afins
1,1019468-32.2019.8.26.0562,Alienação Fiduciária
2,0000043-68.2019.8.26.0616,Furto
3,1007259-02.2018.8.26.0292,DIREITO PREVIDENCIÁRIO
4,1000425-53.2019.8.26.0128,Rural (Art. 48/51)
...,...,...
35022,0001233-88.2018.8.26.0426,Cheque
35023,1514316-66.2019.8.26.0037,Impostos
35024,1008737-24.2019.8.26.0320,Indenização por Dano Material
35025,0000206-22.2019.8.26.0075,IPTU/ Imposto Predial e Territorial Urbano


### Separando grupos de treino e teste de maneira que as distribuições de assuntos permaneçam similares

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
X = documentos_validos.id
y = documentos_validos.Assunto
caminho_corpus = "dados/corpus_tratado/"

#index[0] são os indices de treino, e index[1] são os de teste
#i é o código do experimento
for i, index in enumerate(sss.split(X, y)):    
    exp = i+1
    X_treino, X_teste = X[index[0]], X[index[1]]
    y_treino, y_teste = y[index[0]], y[index[1]]    

### Instanciando o corpus do conjunto de treinamento

In [6]:
from tqdm.notebook import tqdm
import os
diretorio = "dados/corpus_tratado/"
X_treino = pd.DataFrame(X_treino)
X_treino['id'] = X_treino.id + '.txt'

print("criando base de treino para o experimento "+str(exp))
if not os.path.exists('dados/experimento_'+str(exp)):
    os.makedirs('dados/experimento_'+str(exp))

base_treino = open('dados/experimento_'+str(exp)+'/base_treino.txt', 'w+', encoding='utf8')
tokens = 0
for documento in tqdm(X_treino.id.values):
    doc = open(diretorio + documento, 'r', encoding='utf8')
    for frase in doc:
        base_treino.write(frase)
        tokens += len(frase.split(" "))
base_treino.close()
print(str(tokens)+ " tokens copiados com sucesso")
print("preparando documentos para extração do vocabulário:")
X_treino['teores'] = [grouper.recuperar_teor(x, diretorio) for x in tqdm(X_treino.id)]
X_treino['assunto'] = y_treino
X_treino.shape

criando base de treino para o experimento 1



16042036 tokens copiados com sucesso
preparando documentos para extração do vocabulário:


(28021, 3)

### Vocabulário

In [7]:
freq_min = 100
stopwords = nltk.corpus.stopwords.words('portuguese')
vocab = grouper.extrair_vocabulario(X_treino, freq_min, stopwords)

extraindo termos com base no ICA


100%|██████████| 125/125 [00:00<00:00, 331.20it/s]


-processando strings do corpus
-treinando vetorizador
-ICA processado
extraindo termos com base na frequência
extraindo termos do tesauro
extração de vocabulário concluída!


### Treinamento dos modelos

In [9]:
w2v_jur = grouper.treinar_word2vec('dados/experimento_'+str(exp)+'/base_treino.txt', exp)

treinando modelo word2vec


2021-02-23 19:48:59,890 : INFO : collecting all words and their counts
2021-02-23 19:48:59,891 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-23 19:48:59,959 : INFO : PROGRESS: at sentence #10000, processed 235769 words, keeping 13479 word types
2021-02-23 19:49:00,028 : INFO : PROGRESS: at sentence #20000, processed 478506 words, keeping 18827 word types
2021-02-23 19:49:00,100 : INFO : PROGRESS: at sentence #30000, processed 716990 words, keeping 22433 word types
2021-02-23 19:49:00,170 : INFO : PROGRESS: at sentence #40000, processed 956912 words, keeping 25453 word types
2021-02-23 19:49:00,238 : INFO : PROGRESS: at sentence #50000, processed 1198453 words, keeping 27868 word types
2021-02-23 19:49:00,306 : INFO : PROGRESS: at sentence #60000, processed 1434600 words, keeping 30340 word types
2021-02-23 19:49:00,375 : INFO : PROGRESS: at sentence #70000, processed 1678430 words, keeping 32461 word types
2021-02-23 19:49:00,445 : INFO : PROGRESS: 

2021-02-23 19:49:05,525 : INFO : estimated required memory for 32596 words and 100 dimensions: 61932400 bytes
2021-02-23 19:49:05,526 : INFO : resetting layer weights
2021-02-23 19:49:05,851 : INFO : training model with 12 workers on 32596 vocabulary and 100 features, using sg=1 hs=1 sample=1e-05 negative=5 window=5
2021-02-23 19:49:06,863 : INFO : EPOCH 1 - PROGRESS: at 9.06% examples, 327242 words/s, in_qsize 0, out_qsize 3
2021-02-23 19:49:07,864 : INFO : EPOCH 1 - PROGRESS: at 17.97% examples, 326433 words/s, in_qsize 18, out_qsize 3
2021-02-23 19:49:08,865 : INFO : EPOCH 1 - PROGRESS: at 26.88% examples, 328153 words/s, in_qsize 0, out_qsize 0
2021-02-23 19:49:09,867 : INFO : EPOCH 1 - PROGRESS: at 35.51% examples, 326065 words/s, in_qsize 0, out_qsize 0
2021-02-23 19:49:10,875 : INFO : EPOCH 1 - PROGRESS: at 42.69% examples, 314741 words/s, in_qsize 20, out_qsize 1
2021-02-23 19:49:11,883 : INFO : EPOCH 1 - PROGRESS: at 51.44% examples, 316101 words/s, in_qsize 0, out_qsize 0
202

2021-02-23 19:49:45,356 : INFO : EPOCH 4 - PROGRESS: at 15.79% examples, 286306 words/s, in_qsize 0, out_qsize 4
2021-02-23 19:49:46,361 : INFO : EPOCH 4 - PROGRESS: at 22.99% examples, 279527 words/s, in_qsize 22, out_qsize 0
2021-02-23 19:49:47,366 : INFO : EPOCH 4 - PROGRESS: at 31.98% examples, 293010 words/s, in_qsize 7, out_qsize 0
2021-02-23 19:49:48,369 : INFO : EPOCH 4 - PROGRESS: at 40.28% examples, 296115 words/s, in_qsize 0, out_qsize 0
2021-02-23 19:49:49,377 : INFO : EPOCH 4 - PROGRESS: at 47.92% examples, 294113 words/s, in_qsize 0, out_qsize 1
2021-02-23 19:49:50,388 : INFO : EPOCH 4 - PROGRESS: at 56.15% examples, 295054 words/s, in_qsize 17, out_qsize 0
2021-02-23 19:49:51,399 : INFO : EPOCH 4 - PROGRESS: at 65.00% examples, 298588 words/s, in_qsize 0, out_qsize 1
2021-02-23 19:49:52,399 : INFO : EPOCH 4 - PROGRESS: at 72.66% examples, 296972 words/s, in_qsize 18, out_qsize 0
2021-02-23 19:49:53,401 : INFO : EPOCH 4 - PROGRESS: at 81.69% examples, 300459 words/s, in_q

2021-02-23 19:50:26,906 : INFO : EPOCH 7 - PROGRESS: at 60.97% examples, 319637 words/s, in_qsize 0, out_qsize 0
2021-02-23 19:50:27,907 : INFO : EPOCH 7 - PROGRESS: at 68.87% examples, 316294 words/s, in_qsize 14, out_qsize 2
2021-02-23 19:50:28,914 : INFO : EPOCH 7 - PROGRESS: at 76.88% examples, 313770 words/s, in_qsize 0, out_qsize 0
2021-02-23 19:50:29,920 : INFO : EPOCH 7 - PROGRESS: at 84.48% examples, 310036 words/s, in_qsize 0, out_qsize 4
2021-02-23 19:50:30,961 : INFO : EPOCH 7 - PROGRESS: at 92.17% examples, 306444 words/s, in_qsize 22, out_qsize 0
2021-02-23 19:50:31,723 : INFO : worker thread finished; awaiting finish of 11 more threads
2021-02-23 19:50:31,728 : INFO : worker thread finished; awaiting finish of 10 more threads
2021-02-23 19:50:31,734 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-02-23 19:50:31,738 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-02-23 19:50:31,742 : INFO : worker thread finished; awaiting f

2021-02-23 19:51:06,574 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-02-23 19:51:06,575 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-02-23 19:51:06,576 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-02-23 19:51:06,577 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-02-23 19:51:06,579 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-02-23 19:51:06,579 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-02-23 19:51:06,591 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-02-23 19:51:06,602 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-02-23 19:51:06,605 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-02-23 19:51:06,608 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-23 19:51:06,610 : INFO : EPOCH - 10 : training on 15387847 raw words (3696864 effe

In [10]:
ftt_jur = grouper.treinar_fasttext('dados/experimento_'+str(exp)+'/base_treino.txt', exp)

treinando modelo word2vec


2021-02-23 19:51:45,110 : INFO : resetting layer weights
2021-02-23 19:51:50,073 : INFO : collecting all words and their counts
2021-02-23 19:51:50,074 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-23 19:51:50,144 : INFO : PROGRESS: at sentence #10000, processed 235769 words, keeping 13479 word types
2021-02-23 19:51:50,213 : INFO : PROGRESS: at sentence #20000, processed 478506 words, keeping 18827 word types
2021-02-23 19:51:50,283 : INFO : PROGRESS: at sentence #30000, processed 716990 words, keeping 22433 word types
2021-02-23 19:51:50,353 : INFO : PROGRESS: at sentence #40000, processed 956912 words, keeping 25453 word types
2021-02-23 19:51:50,425 : INFO : PROGRESS: at sentence #50000, processed 1198453 words, keeping 27868 word types
2021-02-23 19:51:50,493 : INFO : PROGRESS: at sentence #60000, processed 1434600 words, keeping 30340 word types
2021-02-23 19:51:50,563 : INFO : PROGRESS: at sentence #70000, processed 1678430 words, keeping 324

2021-02-23 19:51:56,048 : INFO : estimated required memory for 32596 words, 138020 buckets and 100 dimensions: 125922344 bytes
2021-02-23 19:51:56,053 : INFO : resetting layer weights
2021-02-23 19:51:59,511 : INFO : training model with 12 workers on 32596 vocabulary and 100 features, using sg=1 hs=1 sample=0.001 negative=5 window=5
2021-02-23 19:52:54,518 : INFO : EPOCH 1 - PROGRESS: at 8.34% words, 16824 words/s, in_qsize -1, out_qsize 1
2021-02-23 19:52:54,519 : INFO : worker thread finished; awaiting finish of 11 more threads
2021-02-23 19:52:54,529 : INFO : worker thread finished; awaiting finish of 10 more threads
2021-02-23 19:52:54,604 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-02-23 19:52:54,715 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-02-23 19:52:55,319 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-02-23 19:52:55,478 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-02-

2021-02-23 19:56:44,205 : INFO : not storing attribute vectors_norm
2021-02-23 19:56:44,208 : INFO : not storing attribute vectors_vocab_norm
2021-02-23 19:56:44,209 : INFO : not storing attribute vectors_ngrams_norm
2021-02-23 19:56:44,209 : INFO : not storing attribute buckets_word
2021-02-23 19:56:44,211 : INFO : storing np array 'vectors_ngrams_lockf' to dados/experimento_1/ftt_jur.model.trainables.vectors_ngrams_lockf.npy
2021-02-23 19:56:51,993 : INFO : saved dados/experimento_1/ftt_jur.model
